#**Comprehensive Analysis of Marketing Strategies:** Regression and Correlation Insights

## 1. Data Loading and Quality Checks

In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Campaign-Data.csv to Campaign-Data.csv


In [5]:
df = pd.read_csv('Campaign-Data.csv')

In [6]:
df.dtypes

,0
Client ID,object
Client Type,object
Number of Customers,int64
Montly Target,int64
Zip Code,int64
Calendardate,object
Amount Collected,int64
Unit Sold,int64
Campaign (Email),float64
Campaign (Flyer),float64


In [13]:
df.head()

,Client ID,Client Type,Number of Customers,Montly Target,Zip Code,Calendardate,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Number of Competition,Month,Year
0,ID-987275,Medium Facility,2800,125,1003,2014-01-16,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low,1,2014
1,ID-987275,Medium Facility,2800,125,1003,2014-02-16,3409460,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,322500.0,Low,2,2014
2,ID-987275,Medium Facility,2800,125,1003,2014-03-18,10228384,75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low,3,2014
3,ID-987275,Medium Facility,2800,125,1003,2014-04-18,17047304,123,0.0,0.0,0.0,0.0,3547500.0,1290000.0,0.0,0.0,Low,4,2014
4,ID-987275,Medium Facility,2800,125,1003,2014-05-19,23866224,171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Low,5,2014


In [7]:
# Creating month, year column

df['Calendardate'] = pd.to_datetime(df['Calendardate'])
df['Month'] = df['Calendardate'].dt.month
df['Year'] = df['Calendardate'].dt.year

<ipython-input-7-c09022bdeb83>:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Calendardate'] = pd.to_datetime(df['Calendardate'])


In [8]:
# Understanding data set

df['Client Type'].value_counts(normalize=True)

,proportion
Client Type,
Large Facility,0.459677
Small Facility,0.282258
Medium Facility,0.169355
Private Facility,0.088710


In [9]:
pd.crosstab(df['Number of Competition'],df['Client Type'],margins=True,normalize='columns')

Client Type,Large Facility,Medium Facility,Private Facility,Small Facility,All
Number of Competition,,,,,
High,0.166667,0.166667,0.166667,0.166667,0.166667
Low,0.833333,0.833333,0.833333,0.833333,0.833333


In [10]:
df.groupby('Number of Competition').mean(numeric_only=True)

,Number of Customers,Montly Target,Zip Code,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Month,Year
Number of Competition,,,,,,,,,,,,,,,
High,1456.935484,75.080645,1003.0,2.974789e+07,213.127016,105398.938508,994046.717540,45198.036895,146945.564516,2.685333e+06,1.786754e+06,72172.379032,8452.620968,10.5,2015.0
Low,1456.935484,75.080645,1003.0,1.445570e+07,103.132258,150862.165766,623692.979839,26693.304194,128219.758065,1.890916e+06,1.883634e+06,70481.854839,15864.919355,5.7,2014.4


Although majority is low competition, most of our sales are coming from high competition markets (Amount Collected).

In [11]:
df.groupby('Client Type').mean(numeric_only=True)

,Number of Customers,Montly Target,Zip Code,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,Sales Contact 3,Sales Contact 4,Sales Contact 5,Month,Year
Client Type,,,,,,,,,,,,,,,
Large Facility,1380.842105,71.578947,1003.0,1.999880e+07,143.098684,142273.609649,8.192056e+05,45595.436623,133667.763158,2.034013e+06,2.017039e+06,119287.280702,16266.447368,6.5,2014.5
Medium Facility,3940.761905,202.857143,1003.0,4.075997e+07,290.583333,437217.097817,1.552603e+06,49176.847619,398645.833333,4.822783e+06,4.698646e+06,85104.166667,33273.809524,6.5,2014.5
Private Facility,400.727273,20.454545,1003.0,5.030246e+06,35.784091,5183.715152,2.272919e+05,5522.470455,1221.590909,6.376705e+05,4.434375e+05,3664.772727,12215.909091,6.5,2014.5
Small Facility,422.514286,21.285714,1003.0,1.637759e+06,11.689286,11975.986310,9.120875e+04,0.000000,8062.500000,7.617143e+05,3.727946e+05,4223.214286,1535.714286,6.5,2014.5


Medium Facility is brining the highest Number of Customers and high sales in Email Campaign. We need to explore this Medium Facility strategy and see if it can be applied to other Facilities.

## 2. Correlation Analysis
### Finding out which variables are highly correlated with 'Amount Collected ($)'

In [21]:
df['Amount Collected'] = pd.to_numeric(df['Amount Collected'], errors='coerce')
df['Amount Collected'] = df['Amount Collected'].fillna(0)
df_cleaned = df.drop(columns=['Client ID', 'Zip Code'])

In [36]:
df_cleaned = pd.get_dummies(df_cleaned, columns=['Client Type'])

In [37]:
df_cleaned = pd.get_dummies(df_cleaned, columns=['Number of Competition'])

In [38]:
df_cleaned.head()

,Number of Customers,Montly Target,Calendardate,Amount Collected,Unit Sold,Campaign (Email),Campaign (Flyer),Campaign (Phone),Sales Contact 1,Sales Contact 2,...,Sales Contact 4,Sales Contact 5,Month,Year,Client Type_Large Facility,Client Type_Medium Facility,Client Type_Private Facility,Client Type_Small Facility,Number of Competition_High,Number of Competition_Low
0,2800,125,2014-01-16,0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1,2014,False,True,False,False,False,True
1,2800,125,2014-02-16,3409460,24,0.0,0.0,0.0,0.0,0.0,...,0.0,322500.0,2,2014,False,True,False,False,False,True
2,2800,125,2014-03-18,10228384,75,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3,2014,False,True,False,False,False,True
3,2800,125,2014-04-18,17047304,123,0.0,0.0,0.0,0.0,3547500.0,...,0.0,0.0,4,2014,False,True,False,False,False,True
4,2800,125,2014-05-19,23866224,171,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5,2014,False,True,False,False,False,True


In [22]:
df_cleaned.dtypes

,0
Client Type,object
Number of Customers,int64
Montly Target,int64
Calendardate,datetime64[ns]
Amount Collected,int64
Unit Sold,int64
Campaign (Email),float64
Campaign (Flyer),float64
Campaign (Phone),float64
Sales Contact 1,float64


In [20]:
df_cleaned.corr()[['Amount Collected']]

,Amount Collected
Number of Customers,0.607496
Montly Target,0.608204
Zip Code,NaN
Calendardate,0.317608
Amount Collected,1.000000
Unit Sold,0.997515
Campaign (Email),0.248235
Campaign (Flyer),0.444337
Campaign (Phone),0.034858
Sales Contact 1,0.277478


In [24]:
# Correlation between variables and Amount Collected

correlation_analysis = pd.DataFrame(df_cleaned[['Amount Collected', 'Campaign (Email)', 'Campaign (Flyer)', 'Campaign (Phone)', 'Sales Contact 1', 'Sales Contact 2', 'Sales Contact 3', 'Sales Contact 4', 'Sales Contact 5']].corr()  ['Amount Collected']).reset_index()
correlation_analysis.columns = ['Impacting Variable','Degree of Linear Impact (r)']
correlation_analysis = correlation_analysis[correlation_analysis['Impacting Variable'] != 'Amount Collected']
correlation_analysis = correlation_analysis.sort_values('Degree of Linear Impact (r)', ascending = False)
correlation_analysis


,Impacting Variable,Degree of Linear Impact (r)
5,Sales Contact 2,0.552112
2,Campaign (Flyer),0.444337
6,Sales Contact 3,0.357887
4,Sales Contact 1,0.277478
1,Campaign (Email),0.248235
7,Sales Contact 4,0.236165
8,Sales Contact 5,0.095795
3,Campaign (Phone),0.034858


In [44]:
df_cleaned.columns = df_cleaned.columns.str.strip()

# Small Facility
small_facility_df = df_cleaned[df_cleaned['Client Type_Small Facility'] == True]

correlation_matrix = small_facility_df[['Amount Collected', 'Campaign (Email)', 'Campaign (Flyer)',
                                        'Campaign (Phone)', 'Sales Contact 1', 'Sales Contact 2',
                                        'Sales Contact 3', 'Sales Contact 4', 'Sales Contact 5']].corr()

correlation_analysis = pd.DataFrame(correlation_matrix['Amount Collected']).reset_index()

correlation_analysis.columns = ['Impacting Variable', 'Degree of Linear Impact (r)']

correlation_analysis = correlation_analysis[correlation_analysis['Impacting Variable'] != 'Amount Collected']
correlation_analysis = correlation_analysis.sort_values('Degree of Linear Impact (r)', ascending=False)
correlation_analysis


,Impacting Variable,Degree of Linear Impact (r)
5,Sales Contact 2,0.219759
6,Sales Contact 3,0.068269
1,Campaign (Email),0.060200
2,Campaign (Flyer),0.040600
7,Sales Contact 4,0.024401
8,Sales Contact 5,0.000927
4,Sales Contact 1,-0.015594
3,Campaign (Phone),NaN


In [50]:
facility_types = ['Client Type_Small Facility', 'Client Type_Medium Facility', 'Client Type_Large Facility', 'Client Type_Private Facility']

all_correlations = []

for facility in facility_types:
    facility_df = df_cleaned[df_cleaned[facility] == True]

    correlation_matrix = facility_df[['Amount Collected', 'Campaign (Email)', 'Campaign (Flyer)',
                                      'Campaign (Phone)', 'Sales Contact 1', 'Sales Contact 2',
                                      'Sales Contact 3', 'Sales Contact 4', 'Sales Contact 5']].corr()

    correlation_analysis = pd.DataFrame(correlation_matrix['Amount Collected']).reset_index()
    correlation_analysis.columns = ['Impacting Variable', 'Degree of Linear Impact (r)']
    correlation_analysis = correlation_analysis[correlation_analysis['Impacting Variable'] != 'Amount Collected']
    correlation_analysis['Facility Type'] = facility.replace('Client Type_', '').strip()
    correlation_analysis = correlation_analysis.sort_values(by='Degree of Linear Impact (r)', ascending=False)
    all_correlations.append(correlation_analysis)

final_correlation_df = pd.concat(all_correlations)

facility_order = ['Small Facility', 'Medium Facility', 'Large Facility', 'Private Facility']
final_correlation_df['Facility Type'] = pd.Categorical(final_correlation_df['Facility Type'], categories=facility_order, ordered=True)
final_correlation_df = final_correlation_df.sort_values(by=['Facility Type', 'Degree of Linear Impact (r)'], ascending=[True, False])
final_correlation_df = final_correlation_df[['Facility Type', 'Impacting Variable', 'Degree of Linear Impact (r)']]
final_correlation_df

,Facility Type,Impacting Variable,Degree of Linear Impact (r)
5,Small Facility,Sales Contact 2,0.219759
6,Small Facility,Sales Contact 3,0.068269
1,Small Facility,Campaign (Email),0.060200
2,Small Facility,Campaign (Flyer),0.040600
7,Small Facility,Sales Contact 4,0.024401
8,Small Facility,Sales Contact 5,0.000927
4,Small Facility,Sales Contact 1,-0.015594
3,Small Facility,Campaign (Phone),NaN
5,Medium Facility,Sales Contact 2,0.509677
2,Medium Facility,Campaign (Flyer),0.453612


## 3.1 Regressions Analysis (Market Sales and Strategies)
### Finding out which variables are statistically significant and its impact on dependent variables - 'Amount Collected ($)'

In [61]:
df_cleaned.columns

Index(['Number_of_Customers', 'Montly_Target', 'Calendardate',
       'Amount_Collected', 'Unit_Sold', 'Campaign_Email', 'Campaign_Flyer',
       'Campaign_Phone', 'Sales_Contact_1', 'Sales_Contact_2',
       'Sales_Contact_3', 'Sales_Contact_4', 'Sales_Contact_5', 'Month',
       'Year', 'Client_Type_Large_Facility', 'Client_Type_Medium_Facility',
       'Client_Type_Private_Facility', 'Client_Type_Small_Facility',
       'Number_of_Competition_High', 'Number_of_Competition_Low'],
      dtype='object')

In [81]:
# Regression Analysis

results = smf.ols(formula='Amount_Collected ~ Campaign_Email + Campaign_Flyer + Campaign_Phone + Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3 + Sales_Contact_4 + Sales_Contact_5', data=df_cleaned).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       Amount_Collected   R-squared:                       0.480
Model:                            OLS   Adj. R-squared:                  0.478
Method:                 Least Squares   F-statistic:                     342.1
Date:                Thu, 12 Sep 2024   Prob (F-statistic):               0.00
Time:                        19:51:54   Log-Likelihood:                -54512.
No. Observations:                2976   AIC:                         1.090e+05
Df Residuals:                    2967   BIC:                         1.091e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        1.481e+06   5.12e+05     

In [83]:
results = smf.ols(formula='Amount_Collected ~ Campaign_Email + Campaign_Flyer + Campaign_Phone + Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3 + Sales_Contact_4 + Sales_Contact_5', data=df_cleaned).fit()

# Extract coefficients and p-values
coefficients = results.params
p_values = results.pvalues

coeff_summary = pd.DataFrame({
    'Coefficient': coefficients,
    'p-value': p_values
}).reset_index()

coeff_summary.columns = ['Variable', 'Coefficient', 'p-value']
print(coeff_summary)

          Variable   Coefficient        p-value
0        Intercept  1.481206e+06   3.865385e-03
1   Campaign_Email  7.932022e-01   1.838816e-01
2   Campaign_Flyer  3.337560e+00   1.017277e-36
3   Campaign_Phone  7.336071e-02   9.444420e-01
4  Sales_Contact_1  4.236796e+00   4.570046e-24
5  Sales_Contact_2  3.638215e+00  8.858664e-155
6  Sales_Contact_3  2.343240e+00   2.639060e-68
7  Sales_Contact_4  1.094783e+01   1.331521e-24
8  Sales_Contact_5  3.507795e+00   4.407166e-01


In [85]:
significant_coefficients = coeff_summary[coeff_summary['p-value'] < 0.05]
print(significant_coefficients)

          Variable   Coefficient        p-value
0        Intercept  1.481206e+06   3.865385e-03
2   Campaign_Flyer  3.337560e+00   1.017277e-36
4  Sales_Contact_1  4.236796e+00   4.570046e-24
5  Sales_Contact_2  3.638215e+00  8.858664e-155
6  Sales_Contact_3  2.343240e+00   2.639060e-68
7  Sales_Contact_4  1.094783e+01   1.331521e-24


In [104]:
import statsmodels.formula.api as smf
import pandas as pd

facility_columns = ['Client_Type_Small_Facility', 'Client_Type_Medium_Facility', 'Client_Type_Large_Facility', 'Client_Type_Private_Facility']

facility_results = []

for facility_col in facility_columns:
    if facility_col not in df_cleaned.columns:
        print(f"Column '{facility_col}' does not exist in DataFrame.")
        continue

    facility_df = df_cleaned[df_cleaned[facility_col] == 1].copy()  # Use .copy() to avoid SettingWithCopyWarning

    results = smf.ols(formula='Amount_Collected ~ Campaign_Email + Campaign_Flyer + Campaign_Phone + Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3 + Sales_Contact_4 + Sales_Contact_5', data=facility_df).fit()

    # Extract coefficients and p-values
    coefficients = results.params
    p_values = results.pvalues

    coeff_summary = pd.DataFrame({
        'Variable': coefficients.index,
        'Coefficient': coefficients.values,
        'p-value': p_values.values
    })

    significant_coefficients = coeff_summary[coeff_summary['p-value'] < 0.05].copy()  # Use .copy() to avoid SettingWithCopyWarning

    significant_coefficients['Facility Type'] = facility_col.replace('Client Type_', '').strip()

    facility_results.append(significant_coefficients)

if facility_results:
    final_facility_results = pd.concat(facility_results, ignore_index=True)
else:
    final_facility_results = pd.DataFrame(columns=['Variable', 'Coefficient', 'p-value', 'Facility Type'])

print(final_facility_results)


           Variable   Coefficient       p-value                 Facility Type
0         Intercept  8.789231e+05  2.362343e-05    Client_Type_Small_Facility
1    Campaign_Phone -3.467699e-06  2.362343e-05    Client_Type_Small_Facility
2   Sales_Contact_2  8.100635e-01  7.940431e-10    Client_Type_Small_Facility
3         Intercept  5.681904e+06  2.544163e-02   Client_Type_Medium_Facility
4    Campaign_Flyer  4.105889e+00  8.893110e-09   Client_Type_Medium_Facility
5   Sales_Contact_1  3.136485e+00  2.204391e-05   Client_Type_Medium_Facility
6   Sales_Contact_2  3.577820e+00  3.717345e-28   Client_Type_Medium_Facility
7   Sales_Contact_3  2.117403e+00  2.784735e-12   Client_Type_Medium_Facility
8         Intercept  2.811701e+06  2.527384e-03    Client_Type_Large_Facility
9    Campaign_Flyer  2.720412e+00  1.937321e-14    Client_Type_Large_Facility
10   Campaign_Phone -3.536085e+00  8.070200e-03    Client_Type_Large_Facility
11  Sales_Contact_1  1.167308e+01  2.968636e-20    Client_Type_L

In [107]:
# Extract significant variables for each facility type
significant_variables = final_facility_results[['Facility Type', 'Variable']]

# Pivot to get a list of significant variables for each facility type
significant_vars_per_facility = significant_variables.pivot(index='Facility Type', columns='Variable', values='Variable')
significant_vars_per_facility = significant_vars_per_facility.notna()
significant_variables

,Facility Type,Variable
0,Client_Type_Small_Facility,Intercept
1,Client_Type_Small_Facility,Campaign_Phone
2,Client_Type_Small_Facility,Sales_Contact_2
3,Client_Type_Medium_Facility,Intercept
4,Client_Type_Medium_Facility,Campaign_Flyer
5,Client_Type_Medium_Facility,Sales_Contact_1
6,Client_Type_Medium_Facility,Sales_Contact_2
7,Client_Type_Medium_Facility,Sales_Contact_3
8,Client_Type_Large_Facility,Intercept
9,Client_Type_Large_Facility,Campaign_Flyer


## 3.2 Running Regression with only significant variables

In [116]:
import pandas as pd
import statsmodels.formula.api as smf

regression_results = {}

# Perform regression for each facility type using only significant variables
for facility in significant_vars_per_facility.index:
    significant_vars = significant_vars_per_facility.columns[significant_vars_per_facility.loc[facility]].tolist()

    # Ensure 'Amount_Collected' is the dependent variable and Intercept, so remove it from independent variables list
    if 'Amount_Collected' in significant_vars:
        significant_vars.remove('Amount_Collected')
    if 'Intercept' in significant_vars:
        significant_vars.remove('Intercept')

    formula = 'Amount_Collected ~ ' + ' + '.join(significant_vars)

    print(f"Formula for {facility}: {formula}")

    facility_df = df_cleaned[df_cleaned[f'{facility}'] == 1].copy()

    # Perform the regression
    try:
        results = smf.ols(formula=formula, data=facility_df).fit()
        regression_results[facility] = results.summary()
    except Exception as e:
        print(f"Error for {facility}: {e}")

for facility, result in regression_results.items():
    print(f"Facility Type: {facility}")
    print(result)
    print("\n" + "="*80 + "\n")


Formula for Client_Type_Large_Facility: Amount_Collected ~ Campaign_Flyer + Campaign_Phone + Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3 + Sales_Contact_4
Formula for Client_Type_Medium_Facility: Amount_Collected ~ Campaign_Flyer + Sales_Contact_1 + Sales_Contact_2 + Sales_Contact_3
Formula for Client_Type_Private_Facility: Amount_Collected ~ Sales_Contact_2
Formula for Client_Type_Small_Facility: Amount_Collected ~ Campaign_Phone + Sales_Contact_2
Facility Type: Client_Type_Large_Facility
                            OLS Regression Results                            
Dep. Variable:       Amount_Collected   R-squared:                       0.370
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     133.5
Date:                Thu, 12 Sep 2024   Prob (F-statistic):          5.80e-133
Time:                        20:54:50   Log-Likelihood:                -25075.
No. Observations:      

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:1967: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


## 4. Final recommendation - Which campaign has the highest ROI and which should we avoid?

From the regression results, we can see how much we can derive from each $1 spend. The highest ROI campaigns are as listed below:

**1. Large Facility**
* Sales_Contact_1 with ROI of $11.65.

* Sales_Contact_4 with ROI of $10.66.

* Sales_Conact_2 with ROI of $4.02.

* Campaign_Phone with negative ROI of -$3.55.

**2. Medium Facility**
* Campaign_Flyer with ROI of $4.28.

* Sales_Contact_2 with ROI of $3.44.

**3. Small Facility**
* Sales_Contact_2 with ROI of $0.84.

**4. Private Facility**
* Sales_Contact_2 with ROI of $7.15.



We can see that Sales contact works very well with Large Facilities while Phone campaigns gives negative effect.

As for Medium and Private Facilities, we can focus on Campaign Flyers and Sales Contacts.

For Small Facilities, we have very low ROI with Sales Contact, we might want to consider a combination of campaigns to optimize ROI.

However these are analysis based on the individual impact. For combination between these tactics, we need further exploration.